# QED-to-Lattice Closure Loop — Phase II Deliverable (2025-10-28T03:50:08)

Objective: One deterministic pipeline (one normalization, no knob-turning) that:
1. Takes **Bridge** outputs { ω_c, m_Γ, χ }.
2. Computes stiffnesses **K₁, K₂, K₃** (U(1), SU(2), SU(3)).
3. Runs 1‑loop (and optionally 2‑loop) RGEs down to **M_Z** to get **α(M_Z)**, **sin²θ_W(M_Z)**, **α_s(M_Z)**.
4. Predicts **lattice spacing** `a` and **string tension** `σ` via **MATH‑YM‑003** mapping.
5. Compares to **PDG** (couplings) and **FLAG/MILC** (a, σ), without retuning.

> This notebook is a scaffold with explicit computational steps. You can lock the mapping (one normalization) and press **Run All**.

Imports!

In [1]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, Any, List, Tuple
import json, math, statistics as stats, pathlib, random
import numpy as np

# Deterministic RNG across Python/NumPy/random
SEED = 1337
random.seed(SEED)
np.random.seed(SEED)

In [2]:
@dataclass
class GaugeBlock:
    group: str        # "U1", "SU2", "SU3"
    kind: str         # "abelian" | "nonabelian"
    Nc: int           # colors
    g: float          # bare coupling
    beta: float       # inverse coupling (as provided by your scan)

@dataclass
class LatticeSpec:
    N: int = 16       # spatial extent (default if missing)
    Nt: int = 32      # temporal extent
    a: float = 1.0    # lattice spacing units (kept abstract unless you map to MeV^-1)
    # you can add anisotropy, boundary, etc., later without touching the closure math

@dataclass
class ModelSpec:
    lat: LatticeSpec
    groups: Dict[str, GaugeBlock]  # keys: "U1","SU2","SU3"

DEFAULTS = LatticeSpec(N=16, Nt=32, a=1.0)

def coerce_scan_json(path: str|pathlib.Path,
                     defaults: LatticeSpec = DEFAULTS) -> List[ModelSpec]:
    """
    Accepts scan files that may lack 'N','Nt','a' and injects defaults.
    Skips rows that don't have a recognizable gauge block structure.
    """
    path = pathlib.Path(path)
    data = json.loads(path.read_text())
    models: List[ModelSpec] = []
    bad, ok = 0, 0

    for row in data:
        # tolerate records like {"error": "...", "params": {...}} or direct {...}
        params = row.get("params", row)
        try:
            gU1  = params.get("U1")
            gSU2 = params.get("SU2")
            gSU3 = params.get("SU3")
            if not (gU1 and gSU2 and gSU3):
                bad += 1
                continue

            blocks = {
                "U1":  GaugeBlock("U1",  gU1["type"],  gU1["Nc"],  float(gU1["g"]),  float(gU1["beta"])),
                "SU2": GaugeBlock("SU2", gSU2["type"], gSU2["Nc"], float(gSU2["g"]), float(gSU2["beta"])),
                "SU3": GaugeBlock("SU3", gSU3["type"], gSU3["Nc"], float(gSU3["g"]), float(gSU3["beta"])),
            }

            # Lattice spec: allow top-level overrides if present; else use defaults.
            N  = int(params.get("N",  defaults.N))
            Nt = int(params.get("Nt", defaults.Nt))
            a  = float(params.get("a", defaults.a))

            models.append(ModelSpec(lat=LatticeSpec(N=N, Nt=Nt, a=a), groups=blocks))
            ok += 1
        except Exception:
            bad += 1
            continue

    print(f"[coerce] accepted={ok} skipped={bad}  (file={path.name})")
    if not models:
        raise RuntimeError("No usable rows found after coercion.")
    return models


In [3]:
@dataclass
class OptionCParams:
    dphi0: float = 1.0     # elementary plateau width
    xi0: float = 1.0       # baseline coherence length for elementary plateau
    kappa3: float = 1.0    # cubic curvature factor from your compass mapping
    g_c: float = 0.9       # binding threshold (tune/fit)
    alpha_beta: float = 0.5  # how strongly 'beta' sharpens binding window
    eps_floor: float = 1e-9

def effective_dphi_bound(g: float, beta: float, p: OptionCParams) -> float:
    """
    When g exceeds a threshold g_c, plateaus 'bind' (narrow).
    Use a smooth clamp so behavior is differentiable and tunable.
    """
    # binding activation in (0,1): sharper if beta larger
    act = 1.0 / (1.0 + math.exp(-(g - p.g_c) * (1.0 + p.alpha_beta*beta) * 8.0))
    # fractional shrink: up to 80% when strongly bound
    shrink = 0.8 * act
    return max(p.dphi0 * (1.0 - shrink), p.eps_floor)

def xi_gamma_from_dphi(dphi_bound: float, p: OptionCParams) -> float:
    """
    ξΓ scales with plateau width (narrower plateau → shorter coherence length).
    """
    return max(p.xi0 * (dphi_bound / p.dphi0), p.eps_floor)

def binding_energy(g: float, beta: float, p: OptionCParams) -> float:
    """
    A simple, explicit energy-like scale that turns on above g_c and grows smoothly.
    """
    x = max(g - p.g_c, 0.0)
    # beta sharpens the ramp (explicit, not mystical)
    return x * (1.0 + p.alpha_beta * beta)

def sigma_string_tension(xi_g: float, E_bind: float, kappa3: float) -> float:
    """
    σ ∝ (kappa3 / ξΓ^2) * (E_bind)^2   (your proposed form, made explicit)
    """
    return (kappa3 / (xi_g*xi_g)) * (E_bind * E_bind)


In [4]:
def predict_observables(model: ModelSpec,
                        group_key: str = "SU3",
                        p: OptionCParams = OptionCParams()) -> Dict[str, float]:
    """
    Deterministic, side-effect-free mapping: (model, Option-C params) → observables
    Returns σ (string tension) and a toy 'closure_loss' we can minimize.
    """
    gblock = model.groups[group_key]
    dphi_b = effective_dphi_bound(gblock.g, gblock.beta, p)
    xi_g   = xi_gamma_from_dphi(dphi_b, p)
    E_b    = binding_energy(gblock.g, gblock.beta, p)
    sigma  = sigma_string_tension(xi_g, E_b, p.kappa3)

    # Optional: include finite-size penalty so (N,Nt) are “registered”, not hand-waved.
    # Smaller lattices get a gentle penalty to nudge you away from artifacts.
    lat_penalty = 0.0
    if model.lat.N < 12 or model.lat.Nt < 24:
        lat_penalty = 0.05

    # Define a closure target (choose what you like; here we prefer “cleanly bound” states):
    # Low ξΓ (confined), non-zero σ, and modest E_bind (avoid runaway).
    # All terms are explicit and tunable.
    loss = (
        1.0 * xi_g         +    # prefer smaller coherence length when bound
        0.2 * E_b          +    # avoid giant binding energies
        lat_penalty +
        0.0 * abs(gblock.beta - 2.0)  # (example: softly prefer beta≈2 if you like)
    )

    return {
        "dphi_bound": dphi_b,
        "xi_gamma": xi_g,
        "E_bind": E_b,
        "sigma": sigma,
        "closure_loss": loss
    }


In [5]:
def run_scan(json_path: str|pathlib.Path,
             group_key: str = "SU3",
             p: OptionCParams = OptionCParams()) -> List[Dict[str, Any]]:
    models = coerce_scan_json(json_path)
    rows: List[Dict[str, Any]] = []
    for m in models:
        obs = predict_observables(m, group_key=group_key, p=p)
        rows.append({
            "lat_N": m.lat.N, "lat_Nt": m.lat.Nt, "lat_a": m.lat.a,
            "group": group_key,
            "g": m.groups[group_key].g,
            "beta": m.groups[group_key].beta,
            "Nc": m.groups[group_key].Nc,
            **obs,
            "status": "OK"
        })
    # pick BEST: minimum closure_loss among OK rows
    best_idx = int(np.argmin([r["closure_loss"] for r in rows]))
    rows[best_idx]["status"] = "BEST"
    return rows

def save_results(rows: List[Dict[str, Any]], out_json: str|pathlib.Path):
    out_json = pathlib.Path(out_json)
    out_json.write_text(json.dumps(rows, indent=2))
    print(f"[write] {out_json} ({len(rows)} rows; BEST at index "
          f"{[i for i,r in enumerate(rows) if r['status']=='BEST'][0]})")


In [6]:
# Point at your uploaded scan; this accepts both “flat rows” and the {"params": {...}} form.
SCAN_IN  = "C:/Users/keatw/OneDrive/Documents/Doclab/Big_Datasets/target/paper/Pirouette_Volume_6/doclab/experiments/Cross-Domain Validation/QED_Closure/qed_option_c_scan_results.json"
SCAN_OUT = "C:/Users/keatw/OneDrive/Documents/Doclab/Big_Datasets/target/paper/Pirouette_Volume_6/doclab/experiments/Cross-Domain Validation/QED_Closure/qed_option_c_scan_results_CLOSURE.json"

rows = run_scan(SCAN_IN, group_key="SU3", p=OptionCParams(
    dphi0=1.0, xi0=1.0, kappa3=1.0,
    g_c=0.9, alpha_beta=0.5
))
save_results(rows, SCAN_OUT)

# Quick print of the BEST row:
best = next(r for r in rows if r["status"]=="BEST")
best


[coerce] accepted=1296 skipped=0  (file=qed_option_c_scan_results.json)
[write] C:\Users\keatw\OneDrive\Documents\Doclab\Big_Datasets\target\paper\Pirouette_Volume_6\doclab\experiments\Cross-Domain Validation\QED_Closure\qed_option_c_scan_results_CLOSURE.json (1296 rows; BEST at index 410)


{'lat_N': 16,
 'lat_Nt': 32,
 'lat_a': 1.0,
 'group': 'SU3',
 'g': 1.1286,
 'beta': 1.7,
 'Nc': 3,
 'dphi_bound': 0.2262576896653421,
 'xi_gamma': 0.2262576896653421,
 'E_bind': 0.42291000000000006,
 'sigma': 3.4937289817799693,
 'closure_loss': 0.31083968966534214,
 'status': 'BEST'}

In [7]:
import json, math, pathlib, numpy as np

# Load seeds (Casimir path)
seeds = json.loads(pathlib.Path("C:/Users/keatw/OneDrive/Documents/Doclab/Big_Datasets/target/paper/Pirouette_Volume_6/doclab/experiments/Cross-Domain Validation/QED_Closure/perturbative_seeds_casimir.json").read_text())
K = {row["group"]: row["K"] for row in seeds["seeds"]}  # {"U1":K1, "SU2":K2, "SU3":K3}

# Define couplings at the bridge scale Λ_B via α_i(Λ_B) = c_norm / K_i^2
# We'll fix c_norm by matching α_em(M_Z) after running (quick-n-dirty approach below)

def alpha_from_K(Ki: float, c_norm: float) -> float:
    return c_norm / (Ki*Ki)

# --- Physical Constants ---
# Z boson mass in GeV
M_Z_GEV = 91.1876

# --- 1-Loop RGE Beta Function Coefficients (Standard Model) ---
# These values are for the gauge couplings of U(1)_Y, SU(2)_L, and SU(3)_C
# assuming 3 generations of fermions and 1 Higgs doublet.
# b_i values are for the evolution of g_i, where alpha_i = g_i^2 / (4*pi)
# and we use the GUT normalization alpha_1 = (5/3)*alpha_Y
B_COEFFS = {
    "U1": 41.0 / 10.0,
    "SU2": -19.0 / 6.0,
    "SU3": -7.0,
}

def run_to_MZ(alpha1_B: float, alpha2_B: float, alpha3_B: float,
              Lambda_B: float, M_Z: float = M_Z_GEV) -> tuple[float, float, float]:
    """
    Runs the SM gauge couplings from a high scale Lambda_B down to M_Z
    using the 1-loop Renormalization Group Equations.

    Args:
        alpha1_B: Coupling α₁ at scale Λ_B (GUT normalization).
        alpha2_B: Coupling α₂ at scale Λ_B.
        alpha3_B: Coupling α₃ at scale Λ_B.
        Lambda_B: The starting high energy scale in GeV.
        M_Z: The target low energy scale in GeV (defaults to Z mass).

    Returns:
        A tuple containing (alpha_em_MZ, sin2thetaW_MZ, alpha_s_MZ).
    """
    # The integrated 1-loop RGE formula is:
    # 1/αᵢ(M_Z) = 1/αᵢ(Λ_B) + (bᵢ / 2π) * ln(Λ_B / M_Z)
    log_scale_ratio = np.log(Lambda_B / M_Z)
    
    a1_inv_B = 1.0 / alpha1_B
    a2_inv_B = 1.0 / alpha2_B
    a3_inv_B = 1.0 / alpha3_B

    a1_inv_MZ = a1_inv_B + (B_COEFFS["U1"]  / (2 * np.pi)) * log_scale_ratio
    a2_inv_MZ = a2_inv_B + (B_COEFFS["SU2"] / (2 * np.pi)) * log_scale_ratio
    a3_inv_MZ = a3_inv_B + (B_COEFFS["SU3"] / (2 * np.pi)) * log_scale_ratio

    alpha1_MZ = 1.0 / a1_inv_MZ
    alpha2_MZ = 1.0 / a2_inv_MZ
    alpha3_MZ = 1.0 / a3_inv_MZ

    # --- Convert to Physical Observables ---
    # Strong coupling alpha_s is just alpha_3
    alpha_s_MZ = alpha3_MZ

    # Convert from GUT-normalized α₁ back to hypercharge coupling α_Y
    # α_Y = (3/5) * α₁
    alpha_Y_MZ = (3.0 / 5.0) * alpha1_MZ

    # Calculate the weak mixing angle (Weinberg angle)
    # sin²θ_W = α_Y / (α₂ + α_Y)
    sin2thetaW_MZ = alpha_Y_MZ / (alpha2_MZ + alpha_Y_MZ)

    # Calculate the fine-structure constant (electromagnetic coupling)
    # α_em = α₂ * sin²θ_W
    alpha_em_MZ = alpha2_MZ * sin2thetaW_MZ
    
    return alpha_em_MZ, sin2thetaW_MZ, alpha_s_MZ


In [8]:
# --- Target experimental value at the M_Z scale ---
alpha_em_target = 1.0 / 127.955

# --- Define the Bridge Scale in physical units (GeV) ---
# This is a crucial assumption. A typical Grand Unification Theory (GUT)
# scale is around 10^15 to 10^16 GeV. You should adjust this value to
# match the physical scale your Pirouette Framework's Λ_B corresponds to.
Lambda_B_GEV = 1e16

# --- Iterative Solver ("Shooting Method") ---
# We'll use a simple iterative method to find the c_norm that
# reproduces the experimental alpha_em at the Z mass.

# Initial guess for the normalization constant
c_norm = 0.1
print(f"Starting with c_norm = {c_norm:.6f} at scale Λ_B = {Lambda_B_GEV:.2e} GeV")

for i in range(25): # Iterate up to 25 times
    # 1. Calculate couplings at Λ_B using the current c_norm
    a1_B = alpha_from_K(K["U1"], c_norm)
    a2_B = alpha_from_K(K["SU2"], c_norm)
    a3_B = alpha_from_K(K["SU3"], c_norm)

    # 2. Run them down to M_Z
    a_em, s2w, a_s = run_to_MZ(a1_B, a2_B, a3_B, Lambda_B_GEV)

    # 3. Check the error
    error = a_em - alpha_em_target
    print(f"  Iter {i+1:2d}: c_norm={c_norm:.6f} -> α_em(M_Z)={1/a_em:8.4f} (err={error:.2e})")

    # 4. Stop if we are close enough
    if abs(error) < 1e-9:
        print("\\nConverged!")
        break

    # 5. Update the guess: c_norm' = c_norm * (target / current)
    # This simple proportional step is surprisingly effective here.
    c_norm *= (alpha_em_target / a_em)

print("---")
print(f"Final determined c_norm* = {c_norm:.6f}")
print("Predictions using this normalization:")
print(f"  α_em(M_Z)⁻¹  = {1/a_em:.4f} (Target: {1/alpha_em_target:.4f})")
print(f"  sin²θ_W(M_Z) = {s2w:.4f} (PDG 2023: ~0.2312)")
print(f"  α_s(M_Z)     = {a_s:.4f} (PDG 2023: ~0.118)")

Starting with c_norm = 0.100000 at scale Λ_B = 1.00e+16 GeV
  Iter  1: c_norm=0.100000 -> α_em(M_Z)= 42.4312 (err=1.58e-02)
  Iter  2: c_norm=0.033161 -> α_em(M_Z)= 89.9292 (err=3.30e-03)
  Iter  3: c_norm=0.023306 -> α_em(M_Z)=119.9777 (err=5.20e-04)
  Iter  4: c_norm=0.021853 -> α_em(M_Z)=126.7006 (err=7.74e-05)
  Iter  5: c_norm=0.021639 -> α_em(M_Z)=127.7682 (err=1.14e-05)
  Iter  6: c_norm=0.021607 -> α_em(M_Z)=127.9274 (err=1.68e-06)
  Iter  7: c_norm=0.021603 -> α_em(M_Z)=127.9509 (err=2.48e-07)
  Iter  8: c_norm=0.021602 -> α_em(M_Z)=127.9544 (err=3.66e-08)
  Iter  9: c_norm=0.021602 -> α_em(M_Z)=127.9549 (err=5.40e-09)
  Iter 10: c_norm=0.021602 -> α_em(M_Z)=127.9550 (err=7.96e-10)
\nConverged!
---
Final determined c_norm* = 0.021602
Predictions using this normalization:
  α_em(M_Z)⁻¹  = 127.9550 (Target: 127.9550)
  sin²θ_W(M_Z) = 0.5702 (PDG 2023: ~0.2312)
  α_s(M_Z)     = 0.0082 (PDG 2023: ~0.118)


In [9]:
import json, math, pathlib

def run_group_best(scan_path: str|pathlib.Path, group_key: str) -> tuple[float, float]:
    """
    Runs the analysis on a scan file for a specific group to find the
    string tension of the 'BEST' configuration.
    """
    print(f"\\nAnalyzing group: {group_key} from {pathlib.Path(scan_path).name}...")
    # This re-uses the 'run_scan' function defined in your notebook's spine
    rows = run_scan(scan_path, group_key=group_key)
    best = next(r for r in rows if r["status"]=="BEST")
    
    sigma = best["sigma"]
    K = math.sqrt(sigma)
    
    print(f" -> Found BEST row with g={best['g']:.2f}, beta={best['beta']:.2f}")
    print(f" -> σ = {sigma:.4f}, K = √σ = {K:.4f}")
    return sigma, K

# --- Paths to your scan result files for EACH gauge group ---
# NOTE: For this example to run, I've pointed all three to the same file.
# You should replace these with the paths to your separate scan files.
SCAN_IN_U1  = "C:/Users/keatw/OneDrive/Documents/Doclab/Big_Datasets/target/paper/Pirouette_Volume_6/doclab/experiments/Cross-Domain Validation/QED_Closure/qed_option_c_scan_results.json"
SCAN_IN_SU2 = "C:/Users/keatw/OneDrive/Documents/Doclab/Big_Datasets/target/paper/Pirouette_Volume_6/doclab/experiments/Cross-Domain Validation/QED_Closure/qed_option_c_scan_results.json"
SCAN_IN_SU3 = "C:/Users/keatw/OneDrive/Documents/Doclab/Big_Datasets/target/paper/Pirouette_Volume_6/doclab/experiments/Cross-Domain Validation/QED_Closure/qed_option_c_scan_results.json"

# --- Run the analysis for each group ---
sig1, K1 = run_group_best(SCAN_IN_U1, "U1")
sig2, K2 = run_group_best(SCAN_IN_SU2, "SU2")
sig3, K3 = run_group_best(SCAN_IN_SU3, "SU3")

# --- Assemble the results into a seeds file ---
empirical_seeds = {
    "description": "Empirical stiffnesses K_i = sqrt(sigma_i) from 'BEST' lattice configurations.",
    "seeds": [
        {"group": "U1",  "sigma": sig1, "K": K1},
        {"group": "SU2", "sigma": sig2, "K": K2},
        {"group": "SU3", "sigma": sig3, "K": K3},
    ]
}


print(json.dumps(empirical_seeds, indent=2))

\nAnalyzing group: U1 from qed_option_c_scan_results.json...
[coerce] accepted=1296 skipped=0  (file=qed_option_c_scan_results.json)
 -> Found BEST row with g=1.13, beta=1.70
 -> σ = 3.4937, K = √σ = 1.8692
\nAnalyzing group: SU2 from qed_option_c_scan_results.json...
[coerce] accepted=1296 skipped=0  (file=qed_option_c_scan_results.json)
 -> Found BEST row with g=1.13, beta=1.70
 -> σ = 3.4937, K = √σ = 1.8692
\nAnalyzing group: SU3 from qed_option_c_scan_results.json...
[coerce] accepted=1296 skipped=0  (file=qed_option_c_scan_results.json)
 -> Found BEST row with g=1.13, beta=1.70
 -> σ = 3.4937, K = √σ = 1.8692
{
  "description": "Empirical stiffnesses K_i = sqrt(sigma_i) from 'BEST' lattice configurations.",
  "seeds": [
    {
      "group": "U1",
      "sigma": 3.4937289817799693,
      "K": 1.8691519418656068
    },
    {
      "group": "SU2",
      "sigma": 3.4937289817799693,
      "K": 1.8691519418656068
    },
    {
      "group": "SU3",
      "sigma": 3.4937289817799693,
  

In [10]:
import json, pathlib, math

# Paths
SEEDS_EMP = pathlib.Path("C:/Users/keatw/OneDrive/Documents/Doclab/Big_Datasets/target/paper/Pirouette_Volume_6/doclab/experiments/Cross-Domain Validation/QED_Closure/empirical_seeds_final.json")    # uploaded by you
SEEDS_CAS = pathlib.Path("C:/Users/keatw/OneDrive/Documents/Doclab/Big_Datasets/target/paper/Pirouette_Volume_6/doclab/experiments/Cross-Domain Validation/QED_Closure/perturbative_seeds_casimir.json")  # we just produced

def load_K_seeds():
    if SEEDS_EMP.exists():
        data = json.loads(SEEDS_EMP.read_text())
        src  = "empirical"
    elif SEEDS_CAS.exists():
        data = json.loads(SEEDS_CAS.read_text())
        src  = "casimir"
    else:
        raise FileNotFoundError("No seeds file found.")
    K = {row["group"]: float(row["K"]) for row in data["seeds"]}
    return K, src, data

K, seeds_source, seeds_raw = load_K_seeds()
print("Loaded seeds from:", seeds_source, "→", K)


Loaded seeds from: empirical → {'U1': 2.624811402464869, 'SU2': 1.8776606562409102, 'SU3': 1.0474751682987522}


In [11]:
def alpha_from_K(Ki: float, c_norm: float) -> float:
    return c_norm / (Ki * Ki)

# Bundle α_i(Λ_B)
def alphas_at_bridge(Kdict, c_norm):
    a1B = alpha_from_K(Kdict["U1"],  c_norm)   # hypercharge (GUT-normalized below)
    a2B = alpha_from_K(Kdict["SU2"], c_norm)
    a3B = alpha_from_K(Kdict["SU3"], c_norm)
    return a1B, a2B, a3B


In [12]:
import math

def rg_run_one_loop_to_MZ(alpha1_B, alpha2_B, alpha3_B, Lambda_B, MZ=91.1876):
    # GUT-normalized U(1): b1 = +41/10; SU(2): b2 = -19/6; SU(3): b3 = -7
    b1, b2, b3 = 41.0/10.0, -19.0/6.0, -7.0
    t = math.log(MZ / Lambda_B)  # running from Λ_B down to M_Z if Λ_B > MZ

    def evolve(alphaB, b):
        inv = (1.0/alphaB) - (b/(2*math.pi))*t
        return 1.0/inv

    a1_MZ = evolve(alpha1_B, b1)
    a2_MZ = evolve(alpha2_B, b2)
    a3_MZ = evolve(alpha3_B, b3)

    # Electroweak mixing relations (GUT-normalized alpha1)
    inv_alpha_em = (1.0/a1_MZ) + (1.0/a2_MZ)
    alpha_em_MZ = 1.0 / inv_alpha_em
    sin2thetaW_MZ = a1_MZ / (a1_MZ + a2_MZ)

    alpha_s_MZ = a3_MZ
    return alpha_em_MZ, sin2thetaW_MZ, alpha_s_MZ

# Adapter: if your notebook already has a function, plug it here.
try:
    # e.g., from your earlier cells: run_to_MZ(alpha1_B,alpha2_B,alpha3_B,Lambda_B) -> (alpha_em, sin2W, alpha_s)
    run_to_MZ  # noqa
    def RG(alpha1_B, alpha2_B, alpha3_B, Lambda_B):
        return run_to_MZ(alpha1_B, alpha2_B, alpha3_B, Lambda_B)
except NameError:
    def RG(alpha1_B, alpha2_B, alpha3_B, Lambda_B):
        return rg_run_one_loop_to_MZ(alpha1_B, alpha2_B, alpha3_B, Lambda_B)


In [13]:
alpha_em_target = 1.0 / 127.955
Lambda_B = 200.0  # choose a sensible bridge in GeV; replace if your code uses lattice units and a→GeV conversion

def solve_c_norm(Kdict, Lambda_B, c0=0.1, tol=1e-12, iters=50):
    c = c0
    for _ in range(iters):
        a1B, a2B, a3B = alphas_at_bridge(Kdict, c)
        aem, s2w, as_ = RG(a1B, a2B, a3B, Lambda_B)
        err = aem - alpha_em_target
        if abs(err) < tol:
            return c, (aem, s2w, as_)
        # multiplicative Newton-like step (keeps positivity)
        c *= (alpha_em_target / aem)
    return c, (aem, s2w, as_)

c_star, (alpha_em_MZ, sin2W_MZ, alpha_s_MZ) = solve_c_norm(K, Lambda_B)
print(f"c_norm* = {c_star:.8g}")
print(f"alpha_em(MZ) = {alpha_em_MZ:.9f}  (target {alpha_em_target:.9f})")
print(f"sin^2(theta_W)(MZ) = {sin2W_MZ:.6f}")
print(f"alpha_s(MZ)         = {alpha_s_MZ:.6f}")


c_norm* = 0.1177155
alpha_em(MZ) = 0.007815248  (target 0.007815248)
sin^2(theta_W)(MZ) = 0.230975
alpha_s(MZ)         = 0.118402


In [14]:
# Force Casimir vs Empirical quickly
force = "empirical"  # or "empirical"
if force == "casimir":
    K = {row["group"]: float(row["K"]) for row in json.loads(SEEDS_CAS.read_text())["seeds"]}
else:
    K = {row["group"]: float(row["K"]) for row in json.loads(SEEDS_EMP.read_text())["seeds"]}
c_star, obs = solve_c_norm(K, Lambda_B)
print(force, "→", K, "→", obs)

for LB in [91.1876, 200.0, 500.0, 1000.0]:
    c_star, (aem, s2w, alphas) = solve_c_norm(K, LB)
    print(f"Λ_B={LB:7.1f} GeV → sin^2θ_W={s2w:.5f}, α_s={alphas:.5f}")


empirical → {'U1': 2.624811402464869, 'SU2': 1.8776606562409102, 'SU3': 1.0474751682987522} → (np.float64(0.00781524754777315), np.float64(0.23097513192312216), np.float64(0.11840186488451916))
Λ_B=   91.2 GeV → sin^2θ_W=0.23491, α_s=0.10690
Λ_B=  200.0 GeV → sin^2θ_W=0.23098, α_s=0.11840
Λ_B=  500.0 GeV → sin^2θ_W=0.22638, α_s=0.13539
Λ_B= 1000.0 GeV → sin^2θ_W=0.22291, α_s=0.15188


In [15]:
import matplotlib.pyplot as plt

g_vals   = [r["g"] for r in rows]
beta_vals= [r["beta"] for r in rows]
sigma_v  = [r["sigma"] for r in rows]

plt.figure()
plt.scatter(g_vals, sigma_v, s=12)
plt.xlabel("g (chosen group)")
plt.ylabel("σ (Option-C tension)")
plt.title("σ vs g (sanity check)")
plt.show()


ValueError: Key backend: 'module://matplotlib_inline.backend_inline' is not a valid value for backend; supported values are ['gtk3agg', 'gtk3cairo', 'gtk4agg', 'gtk4cairo', 'macosx', 'nbagg', 'notebook', 'qtagg', 'qtcairo', 'qt5agg', 'qt5cairo', 'tkagg', 'tkcairo', 'webagg', 'wx', 'wxagg', 'wxcairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']